# **Book Recommendation System**

_Ugo Nuela Jeremiah_

## Importing required Libraries

In [6]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
en_stopwords = stopwords.words('english')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nuela\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nuela\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Loading of Dataset

In [8]:
books = pd.read_csv(r"C:\Users\Nuela\Downloads\Book Recommend\Books.csv")
ratings = pd.read_csv(r"C:\Users\Nuela\Downloads\Book Recommend\Ratings.csv")
users = pd.read_csv(r"C:\Users\Nuela\Downloads\Book Recommend\Users.csv")

C:\Users\Nuela\AppData\Local\Temp\ipykernel_4104\863137209.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(r"C:\Users\Nuela\Downloads\Book Recommend\Books.csv")


## Data Wrangling

In [9]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [10]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [11]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


## Shape of the data

In [12]:
print("Books Shape is: \n", books.shape)
print("Ratings Shape is: \n", ratings.shape)
print("Users Shape is: \n", users.shape)

Books Shape is: 
 (271360, 8)
Ratings Shape is: 
 (1149780, 3)
Users Shape is: 
 (278858, 3)


## Check for duplicates

In [13]:
print("Duplicates in books: \n", books.duplicated().sum())
print("Duplicates in ratings: \n", ratings.duplicated().sum())
print("Duplicates in users: \n", users.duplicated().sum())

Duplicates in books: 
 0
Duplicates in ratings: 
 0
Duplicates in users: 
 0


## Check for missing values

In [14]:
print("Total null in Books: \n", books.isnull().sum())
print("Total null in ratings: \n", ratings.isnull().sum())
print("Total null in users: \n", users.isnull().sum())

Total null in Books: 
 ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64
Total null in ratings: 
 User-ID        0
ISBN           0
Book-Rating    0
dtype: int64
Total null in users: 
 User-ID          0
Location         0
Age         110762
dtype: int64


In [15]:
df_books = books.merge(ratings,on="ISBN")
df_books.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0


In [16]:
df = df_books.copy()

In [17]:
df.dropna(inplace=True) # drop rows with NaN values

In [18]:
df.drop(columns=["ISBN","Year-Of-Publication","Image-URL-S","Image-URL-M", "Image-URL-L", "User-ID"],axis=1,inplace=True)
df.head(3)

,Book-Title,Book-Author,Publisher,Book-Rating
0,Classical Mythology,Mark P. O. Morford,Oxford University Press,0
1,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,5
2,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,0


In [19]:
df['text'] = df['Book-Title'] + " " + df['Book-Author'] + " " + df['Publisher']


In [20]:
df.text

0          Classical Mythology Mark P. O. Morford Oxford ...
1          Clara Callan Richard Bruce Wright HarperFlamin...
2          Clara Callan Richard Bruce Wright HarperFlamin...
3          Clara Callan Richard Bruce Wright HarperFlamin...
4          Clara Callan Richard Bruce Wright HarperFlamin...
                                 ...                        
1031131    There's a Bat in Bunk Five Paula Danziger Rand...
1031132      From One to One Hundred Teri Sloat Dutton Books
1031133    Lily Dale : The True Story of the Town that Ta...
1031134    Republic (World's Classics) Plato Oxford Unive...
1031135    A Guided Tour of Rene Descartes' Meditations o...
Name: text, Length: 1031129, dtype: object

In [21]:
df.drop(columns = ["Book-Author", "Book-Title", "Publisher", "Book-Rating"], inplace=True)
df

,text
0,Classical Mythology Mark P. O. Morford Oxford ...
1,Clara Callan Richard Bruce Wright HarperFlamin...
2,Clara Callan Richard Bruce Wright HarperFlamin...
3,Clara Callan Richard Bruce Wright HarperFlamin...
4,Clara Callan Richard Bruce Wright HarperFlamin...
...,...
1031131,There's a Bat in Bunk Five Paula Danziger Rand...
1031132,From One to One Hundred Teri Sloat Dutton Books
1031133,Lily Dale : The True Story of the Town that Ta...
1031134,Republic (World's Classics) Plato Oxford Unive...


## __Preprocessing Pipepline__
1. Punctuation Removal and Stop Word removal
2. Lower Case Conversion
3. Tokenization
4. Stem

In [22]:
def clean(text):
    re.sub("[^A-Za-z1-9 ]", "", text)
    text = text.lower()
    tokens = word_tokenize(text)
    clean_list = []
    for token in tokens:
        if token not in en_stopwords:
            clean_list.append(stemmer.stem(token))
    return " ".join(clean_list)

In [23]:
df.text = df.text.apply(clean)

In [24]:
df.text

0          classic mytholog mark p. o. morford oxford uni...
1          clara callan richard bruce wright harperflamin...
2          clara callan richard bruce wright harperflamin...
3          clara callan richard bruce wright harperflamin...
4          clara callan richard bruce wright harperflamin...
                                 ...                        
1031131    's bat bunk five paula danzig random hous chil...
1031132                 one one hundr teri sloat dutton book
1031133    lili dale : true stori town talk dead christin...
1031134    republ ( world 's classic ) plato oxford unive...
1031135    guid tour rene descart ' medit first philosoph...
Name: text, Length: 1031129, dtype: object

## __Feature Extraction__

In [34]:
vectorizer = CountVectorizer()

## Term Frequency - Inverse Document Frequency (TF-IDF)

In [25]:
vectorizer = TfidfVectorizer()

In [26]:
text_matrix = vectorizer.fit_transform(df.text)

In [31]:
text_matrix

<1031129x99334 sparse matrix of type '<class 'numpy.float64'>'
	with 7908883 stored elements in Compressed Sparse Row format>

## K-nearest Neighbor

In [37]:
nn = NearestNeighbors()

In [38]:
nn.fit(text_matrix)

NearestNeighbors()

In [39]:
def inference(text):
    text = clean(text)
    text_matrix = vectorizer.transform([text])
    return nn.kneighbors(n_neighbors=5, X = text_matrix, return_distance = False)

In [52]:
test_text = df.text[80]

In [53]:
inference(test_text)

array([[    80,     81, 876788, 876789, 712906]], dtype=int64)

In [54]:
#View the closest 5 neighbors
df.iloc[[    80,     81, 876788, 876789, 712906]].text

80           middl stori sheila heti hous anansi press
81           middl stori sheila heti hous anansi press
876790         happen michael winter hous anansi press
876791         happen michael winter hous anansi press
712906    circl game margaret atwood hous anansi press
Name: text, dtype: object

In [58]:
cosine_sim = cosine_similarity(text_matrix)
cosine_sim

MemoryError: Unable to allocate 1.27 TiB for an array with shape (174545563115,) and data type int64